In [3]:
import sys
import os
CURR_PATH = f'/home/wd/Documents/work_stuff/ViT_REPLICATION'
sys.path.append(os.path.abspath(CURR_PATH))  # Adds root directory to sys.path

# Model Architecture and Data Loading

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# config numbers
class ViTConfig:
    def __init__(self,
                 img_size=32,
                 patch_size=4,
                 in_channels=3,
                 emb_size=64,
                 depth=6,
                 num_heads=4,
                 mlp_ratio=4.0,
                 num_classes=10,
                 dropout=0.1):
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.emb_size = emb_size
        self.depth = depth
        self.num_heads = num_heads
        self.mlp_ratio = mlp_ratio
        self.num_classes = num_classes
        self.dropout = dropout

# Patch Embedding
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, patch_size, emb_size, img_size):
        super().__init__()
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.projection = nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.projection(x)        # (B, emb_size, H/patch, W/patch)
        x = x.flatten(2)              # (B, emb_size, N)
        x = x.transpose(1, 2)         # (B, N, emb_size)
        return x                      # Shape: (B, N, E)

# Transformer Encoder Block
class TransformerEncoderBlock(nn.Module):
    def __init__(self, emb_size, num_heads, mlp_ratio, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(emb_size)
        self.attn = nn.MultiheadAttention(embed_dim=emb_size, num_heads=num_heads, batch_first=True)
        self.ln2 = nn.LayerNorm(emb_size)
        self.mlp = nn.Sequential(
            nn.Linear(emb_size, int(emb_size * mlp_ratio)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(emb_size * mlp_ratio), emb_size),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x), self.ln1(x), self.ln1(x))[0]
        x = x + self.mlp(self.ln2(x))
        return x

# Vit Test Model
class VisionTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        cfg = config["model"]
        
        self.CHANNEL = cfg["in_channels"]
        self.PATCH = cfg["patch_size"]
        self.EMBEDDING = cfg["emb_size"]
        self.IMAGE = cfg["img_size"]
        self.NUM_HEADS = cfg["num_heads"]
        self.MLP_RATIO = cfg["mlp_ratio"]
        self.DROPOUT = cfg["dropout"]
        self.NUM_CLASS = cfg["num_classes"]
        self.DEPTH = cfg["depth"]

        self.patch_embed = PatchEmbedding(
            in_channels=self.CHANNEL,
            patch_size=self.PATCH,
            emb_size=self.EMBEDDING,
            img_size=self.IMAGE
        )

        self.n_patches = (self.IMAGE// self.PATCH) ** 2
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.EMBEDDING))
        self.pos_embedding = nn.Parameter(torch.randn(1, self.n_patches + 1, self.EMBEDDING))

        self.encoder = nn.Sequential(*[
            TransformerEncoderBlock(
                emb_size=self.EMBEDDING,
                num_heads=self.NUM_HEADS,
                mlp_ratio=self.MLP_RATIO,
                dropout=self.DROPOUT
            ) for _ in range(self.DEPTH)
        ])

        self.norm = nn.LayerNorm(self.EMBEDDING)
        self.head = nn.Linear(self.EMBEDDING, self.NUM_CLASS)

    def forward(self, x):
        B = x.size(0)
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embedding[:, :x.size(1), :]
        x = self.encoder(x)
        cls_out = self.norm(x[:, 0])
        return self.head(cls_out)


In [4]:
from utils.config_loader import load_config
from utils.data_loader import DatasetLoader

# loading config file for model
config = load_config(f"{CURR_PATH}/config/vit_test_config.yaml")
model = VisionTransformer(config)

print('testing model sanity')
dummy_input = torch.randn(1, 3, 32, 32)
print(dummy_input.shape)
with torch.no_grad():
    output = model(dummy_input)

print("Output shape:", output.shape)  # torch.Size([1, 10])


print('loading training testing data')
# loading config file for CIFAR10
data_cfg = config["data"]
DATASET = data_cfg["dataset"]
DATA_DIR = data_cfg["data_path"]
BATCH = data_cfg["batch_size"]
NUM_WORKERS = data_cfg["num_workers"]
IMAGE = data_cfg["img_size"]

# loading data
loader = DatasetLoader(dataset_name=DATASET,
                        data_dir=DATA_DIR,
                        batch_size=BATCH,
                        num_workers=NUM_WORKERS,
                        img_size=IMAGE)
train_loader, test_loader = loader.get_loaders()
print(f"Train batches: {len(train_loader)}, Test batches: {len(test_loader)}")
print('data sanity check')
for images, labels in train_loader:
    print(f'image shape and labels shape in training data - one batch : {images.shape}, {labels.shape}')
    break

testing model sanity
torch.Size([1, 3, 32, 32])
Output shape: torch.Size([1, 10])
loading training testing data
Train batches: 782, Test batches: 157
data sanity check
image shape and labels shape in training data - one batch : torch.Size([64, 3, 32, 32]), torch.Size([64])


# Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from models.vit import VisionTransformer
from utils.config_loader import load_config
from utils.data_loader import DataLoaderFactory


def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = VisionTransformer(config).to(device)

cuda


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config["train"]["lr"])

# Training loop
for epoch in range(config["train"]["epochs"]):
    print(f"\nEpoch {epoch+1}/{config['train']['epochs']}")

    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
